In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/tf/notebooks/src


## Intialization

### GPU

In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Packages

In [4]:
!pip install pandas iterative-stratification nlpaug==0.0.20 tqdm click tensorflow_probability==0.11.1 tf2_resnets tensorflow_addons==0.11.1

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### Imports

In [5]:
import os
import random
import logging
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

from sklearn.model_selection import GroupKFold

from dataloaders.train import BalancedMelSampler
from dataloaders.val import MelSampler
from dataloaders.utils import csv_to_dict

from losses import NpairsLoss, MovingAverageBCE
from metrics import TFLWLRAP
from split_data import get_split
from train import get_model, get_callbacks
from models import NUM_FRAMES, Classifier, DeepMetricLearning

In [6]:
from params import *
from utils.logger import prepare_log_folder, create_logger

### Setup

In [7]:
os.environ["TF_DETERMINISTIC_OPS"] = "1"
os.environ["SM_FRAMEWORK"] = "tf.keras"

physical_devices = tf.config.list_physical_devices("GPU")
for i in range(len(physical_devices)):
    tf.config.experimental.set_memory_growth(physical_devices[i], True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

## Main

In [9]:
def main(fold_idx, saved_path):
    os.makedirs(os.path.join(saved_path, f"fold{fold_idx}"), exist_ok=True)

    print(' -> Preparing Data \n')
    
    train_data = pd.read_csv("../data/new_train_tp.csv")
#     train_index, val_index = get_split(fold=fold_idx)
    
    skf = GroupKFold(n_splits=5)
    train_index, val_index = list(skf.split(
        X=train_data, 
        groups=train_data['recording_id'].apply(lambda x:x.split('_')[0])
    ))[fold_idx]
    
    
    fold_train_dict = csv_to_dict(train_data.iloc[train_index])
    fold_valid_dict = csv_to_dict(train_data.iloc[val_index])

    balanced_train_data_loader = BalancedMelSampler(
        fold_train_dict,
        batch_size=64,
        max_length=NUM_FRAMES,
        n_classes=24,
        use_cutmix=True,
        n_classes_in_batch=8,
    )

    valid_data_loader = MelSampler(
        fold_valid_dict,
        batch_size=balanced_train_data_loader.batch_size,
        n_classes=balanced_train_data_loader.n_classes,
        max_length=NUM_FRAMES,
    )

    print(' -> Preparing Model \n')
    model = get_model(
        saved_path=saved_path,
        pretrained_with_contrastive=True,
    )
    model._build()
    
    scheduler = tfa.optimizers.Triangular2CyclicalLearningRate(
        initial_learning_rate=1e-4,
        maximal_learning_rate=1e-3,
        step_size=50,
    )
    
    optimizer = tf.keras.mixed_precision.experimental.LossScaleOptimizer(
        tfa.optimizers.Lookahead(
            tf.keras.optimizers.Adam(learning_rate=scheduler),
            10,
            0.5,
        ),
        "dynamic",
    )
    
    step_per_epoch = int((len(fold_train_dict)) / balanced_train_data_loader.batch_size)
    ma_bce = MovingAverageBCE(
        train_data.iloc[train_index],
        start_apply_step=20 * step_per_epoch,
        momentum=0.9,
        name="moving_average_loss",
    )
    
    model.compile(
        optimizer=optimizer,
        metrics=[TFLWLRAP(num_classes=24, name="lwlrap")],
        metric_loss_fn=NpairsLoss(temperature=0.1, name="n_pairs"),
        classification_loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        moving_average_bce=ma_bce,
    )   

    print(' -> Training Model \n')

    callbacks = get_callbacks(True, fold_idx, saved_path=saved_path)
    steps_per_epoch = int((len(fold_train_dict)) / balanced_train_data_loader.batch_size)
    
    model.fit(
        balanced_train_data_loader,
        steps_per_epoch=steps_per_epoch,
        epochs=100,
        validation_data=valid_data_loader,
        callbacks=callbacks,
        verbose=2
    )

In [10]:
DEBUG = False
pretrained_folder = "../logs/2021-01-28/1/"   # simple
pretrained_folder = "../logs/2021-01-30/42/"  # mixstyle + iunterpolate + cbam
log_folder = "../logs/"

In [11]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f'Logging results to {log_folder}')
    create_logger(directory=log_folder, name="logs.txt")
    
for fold_idx in range(5):
    tf.keras.backend.clear_session()
    print(f"\n-------------   Fold {fold_idx + 1} / {5}  -------------\n")
    
    main(fold_idx,  log_folder)
    
    if DEBUG:
        break

Logging results to ../logs/2021-02-06/9/

-------------   Fold 1 / 5  -------------

 -> Preparing Data 

 -> Preparing Model 

 -> Training Model 

Epoch 1/100
Instructions for updating:
Use `tf.linalg.matmul` instead
15/15 - 7s - loss: 3.9791 - val_loss: 3.2878
Epoch 2/100
15/15 - 6s - loss: 3.8870 - val_loss: 3.3340
Epoch 3/100
15/15 - 6s - loss: 3.7101 - val_loss: 3.3092
Epoch 4/100
15/15 - 6s - loss: 3.1733 - val_loss: 3.0383
Epoch 5/100
15/15 - 6s - loss: 3.1954 - val_loss: 3.0084
Epoch 6/100
15/15 - 6s - loss: 3.8903 - val_loss: 2.9733
Epoch 7/100
15/15 - 6s - loss: 4.0556 - val_loss: 2.8044
Epoch 8/100
15/15 - 6s - loss: 2.8697 - val_loss: 2.5725
Epoch 9/100
15/15 - 5s - loss: 4.0475 - val_loss: 2.7128
Epoch 10/100
15/15 - 6s - loss: 2.9665 - val_loss: 2.2989
Epoch 11/100
15/15 - 6s - loss: 2.8749 - val_loss: 2.4549
Epoch 12/100
15/15 - 6s - loss: 2.7849 - val_loss: 2.3368
Epoch 13/100
15/15 - 6s - loss: 3.6961 - val_loss: 2.3224
Epoch 14/100
15/15 - 6s - loss: 4.1030 - val_los

Epoch 54/100
15/15 - 6s - loss: 3.6256 - val_loss: 2.1408
Epoch 55/100
15/15 - 6s - loss: 3.5476 - val_loss: 2.1755
Epoch 56/100
15/15 - 6s - loss: 3.5884 - val_loss: 2.0897
Epoch 57/100
15/15 - 6s - loss: 3.6720 - val_loss: 2.2294
Epoch 58/100
15/15 - 6s - loss: 2.1102 - val_loss: 2.1116
Epoch 59/100
15/15 - 5s - loss: 3.9546 - val_loss: 2.1519
Epoch 60/100
15/15 - 6s - loss: 3.4935 - val_loss: 2.1561
Epoch 61/100
15/15 - 6s - loss: 2.1375 - val_loss: 2.1947
Epoch 62/100
15/15 - 6s - loss: 2.2258 - val_loss: 2.1222
Epoch 63/100
15/15 - 6s - loss: 2.2177 - val_loss: 2.0663
Epoch 64/100
15/15 - 6s - loss: 2.1392 - val_loss: 2.1054
Epoch 65/100
15/15 - 6s - loss: 2.1742 - val_loss: 2.1208
Epoch 66/100
15/15 - 6s - loss: 3.6244 - val_loss: 2.0282
Epoch 67/100
15/15 - 6s - loss: 2.2577 - val_loss: 2.0094
Epoch 68/100
15/15 - 6s - loss: 2.1580 - val_loss: 2.0315
Epoch 69/100
15/15 - 6s - loss: 2.1167 - val_loss: 2.1095
Epoch 70/100
15/15 - 6s - loss: 2.2247 - val_loss: 2.0571
Epoch 71/100
1

Epoch 41/100
15/15 - 5s - loss: 3.9023 - val_loss: 2.1673
Epoch 42/100
15/15 - 5s - loss: 3.6525 - val_loss: 2.1568
Epoch 43/100
15/15 - 6s - loss: 3.6492 - val_loss: 2.2604
Epoch 44/100
15/15 - 6s - loss: 2.3557 - val_loss: 2.2052
Epoch 45/100
15/15 - 5s - loss: 2.4357 - val_loss: 2.2862
Epoch 46/100
15/15 - 5s - loss: 2.3011 - val_loss: 2.2119
Epoch 47/100
15/15 - 6s - loss: 2.3628 - val_loss: 2.2191
Epoch 48/100
15/15 - 5s - loss: 3.9422 - val_loss: 2.1657
Epoch 49/100
15/15 - 6s - loss: 2.1781 - val_loss: 2.0902
Epoch 50/100
15/15 - 6s - loss: 3.7182 - val_loss: 2.0668
Epoch 51/100
15/15 - 5s - loss: 3.6149 - val_loss: 2.1131
Epoch 52/100
15/15 - 6s - loss: 2.2386 - val_loss: 2.1233
Epoch 53/100
15/15 - 5s - loss: 3.5864 - val_loss: 2.1179
Epoch 54/100
15/15 - 6s - loss: 3.9203 - val_loss: 2.0863
Epoch 55/100
15/15 - 5s - loss: 3.6085 - val_loss: 2.0910
Epoch 56/100
15/15 - 6s - loss: 3.5860 - val_loss: 2.0618
Epoch 57/100
15/15 - 5s - loss: 2.2085 - val_loss: 2.1117
Epoch 58/100
1